<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/LangChain_OpenAI_Moderation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">LLM Hands On Course</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Create a Moderation system with LangChain and OpenAI.</h2>
    <p>by <b>Pere Martra</b></p>
</div>

<br>

<div align="center">
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

# How To Create a Moderation System Using LangChain.

We are goin to create a Moderation System based in two Models. The first Model  reads the User comments and answer them.

The second language Model receives the answer of the first model and identify any kind on negativity modifiyng if necessary the comment.

With the intention of preventing a text entry by the user from influencing a negative or out-of-tone response from the comment system.

In [ ]:
#Install de LangChain and openai libraries.
!pip install -q langchain==0.1.4
!pip install -q langchain-openai==0.0.5


## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.
* OpenAI:  To interact with the OpenAI models.


In [ ]:
#PrompTemplate is a custom class that provides funcrionality to create prompts
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


In [ ]:
import torch
import os
import numpy as np

We need an OpenAI key to interac with the OpenAI API.

Here you can acces to your keys.
https://platform.openai.com/account/api-keys

OpenAI it's a pay service and you need a credit card to get a Key. But is a a relly cheap service if you only want to do some test like the ones in this notebook.

I'm using the gpt-3.5 as a moderator.


In [ ]:
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")


OpenAI API Key: ··········


In [ ]:
assistant_llm = ChatOpenAI(model="gpt-3.5-turbo")

Create the template for the first model called **assistant**.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment.

I included the sentiment to facilitate the creation of rude or incorrect answers.

In [ ]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} assistant that responds to user comments,
using similar vocabulary than the user.
User:" {customer_request}"
Comment:
"""

In [ ]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

output_parser = StrOutputParser()

Now we create a First Chain. Just chaining the assistant_prompt_template and the model. The model will receive the prompt generated with the prompt_template.  

In [ ]:
#assistant_chain = LLMChain(
#    llm=assistant_llm,
#    prompt=assistant_prompt_template,
#    output_key="assistant_response",
#    verbose=False,
#)

assistant_chain = assistant_prompt_template | assistant_llm | output_parser

#the output of the formatted prompt will pass directly to the LLM.

To execute the chain created it's necessary to call the **.run** method of the chain, and pass the variables necessaries.

In our case: *customer_request* and *sentiment*.

In [ ]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #calling the .invoke method from the chain created Above.
    assistant_response = assistant_chain.invoke(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model, and to obtain it we are changing the sentiment.

In [ ]:
# This is the customer request, or customer comment in the forum moderated by the agent.
# feel free to modify it.
customer_request = """This product is a piece of shit. I feel like an Idiot!"""


In [ ]:
# Our assistant working in 'nice' mode.
response_data=create_dialog(customer_request, "nice")
print(f"assistant response: {response_data}")

assistant response: I totally get where you're coming from. It's frustrating when a product doesn't live up to our expectations. It can make us feel pretty foolish for investing in it.


The answer obtained is really polite. It dosn't need moderation.

In [ ]:
#Our assistant running in rude mode.
response_data = create_dialog(customer_request, "most rude possible ")
print(f"assistant response: {response_data}")

assistant response: Wow, you really hit the nail on the head with that one! This product is indeed a total piece of shit. And hey, don't worry, feeling like an idiot is completely justified when dealing with such a worthless item.


As you can see the answers we obtain are not polite and we can't publish this messages to the forum, especially if they come from our company's AI assistant.

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.


In [ ]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will receive a Original comment and if it is impolite you must transform in polite.
Try to mantain the meaning when possible,

If it it's polite, you will let it remain as is and repeat it word for word.
Original comment: {comment_to_moderate}
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template,
)

In [ ]:
#I'm going to use a more advanced LLM
moderator_llm = ChatOpenAI(model="gpt-4")

In [ ]:
#We build the chain for the moderator.
#moderator_chain = LLMChain(
#    llm=moderator_llm, prompt=moderator_prompt_template, verbose=False
#)  # the output of the prompt will pass to the LLM.

moderator_chain = moderator_prompt_template | moderator_llm | output_parser

In [ ]:
# To run our chain we use the .run() command
moderator_data = moderator_chain.invoke({"comment_to_moderate": response_data})
print(moderator_data)

Wow, you really hit the nail on the head with that one! This product indeed leaves much to be desired. And hey, don't worry, feeling disappointed is completely justified when dealing with such an unsatisfactory item.


Maybe the message is not perfect, but for sure that is more polite than the one produced by the ***rude assistant***.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.


In [ ]:
assistant_moderated_chain = (
    {"comment_to_moderate":assistant_chain}
    |moderator_chain
)


**SequentialChain** is used to link different chains and parameters.

It's necessary to indicate the chains and the parameters that we shoud pass in the **.invoke** method.

In [ ]:
#from langchain.chains import SequentialChain

# Creating the SequentialChain class indicating chains and parameters.
#assistant_moderated_chain = SequentialChain(
#    chains=[assistant_chain, moderator_chain],
#    input_variables=["sentiment", "customer_request"],
#    verbose=True
#)

In [ ]:
# We can now run the chain.
from langchain.callbacks.tracers import ConsoleCallbackHandler
assistant_moderated_chain.invoke({"sentiment": "nice", "customer_request": customer_request},
                                 config={'callbacks':[ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "nice",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate>] Entering Chain run with input:
{
  "sentiment": "nice",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "nice",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence > 4:prompt:PromptTemplate] Entering Prompt run with input:
{
  "sentiment": "nice",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParalle

'I completely resonate with your perspective, this product unfortunately did not meet my expectations. I regret making the purchase.'

Lets use our Moderating System!

In [ ]:
# We can now run the chain.
assistant_moderated_chain.invoke({"sentiment": "impolite", "customer_request": customer_request},
                                 config={'callbacks':[ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate>] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<comment_to_moderate> > 3:chain:RunnableSequence > 4:prompt:PromptTemplate] Entering Prompt run with input:
{
  "sentiment": "impolite",
  "customer_request": "This product is a piece of shit. I feel like an Idiot!"
}
[chain/end] [1:chain:RunnableSequence > 2:chain

"Thank you for your feedback. This product might not meet your expectations. It may not be the most suitable choice for everyone, but it's important to remember that different people have different preferences."

Every time you execute this function you can get different messages, but for sure than the one in the ***Finished Chain*** generated by our ***moderator*** is more suitable than the one in Original Comment generated by our ***rude assistant***.